# Dataset Features
The training data contains the following engineered and raw features:

- **MAF**
- **R2**
- **het**
- **platform**
- **cluster**
- **GERP_RS**
- **GERP_NR**
- **GERP_RS_rankscore**
- **gnomADe_AF**
- **synonymous**
- **SIFT**
- **gnomADg_AF**
- **IMPACT**
- **protein_coding**
- **LCR** (True if in LCR, else False)
- **type** (Genotyped / Imputed)

In [ ]:
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from imblearn.over_sampling import RandomOverSampler
import gc
import xgboost, catboost, lightgbm

In [ ]:
#Data loading 
df = pd.read_csv('data.csv') # described above

# Derived variables
df['synonymous']   = df['CSQ'] == 'synonymous_variant'
df['protein_coding'] = df['BIOTYPE'] == 'protein_coding'
df['het'] = df['het.x'] / (df['hom_alt.x'] + df['hom_ref.x'] + df['het.x'])

# Target variable
y_clf = (df['p_flag'] == 'Discordant').astype(int)

FEATURES = ['MAF','R2','het','platform','cluster','GERP_RS','GERP_NR','GERP_RS_rankscore',
            'gnomADe_AF','synonymous','SIFT','gnomADg_AF','IMPACT','protein_coding','LCR','type']

# One‑hot encode categoricals
X = pd.get_dummies(df[FEATURES], drop_first=True)

# Train / test split with oversampling on training fold
ros = RandomOverSampler(random_state=666)
stratifier = df[['p_flag','platform','cluster']].astype(str).agg('-'.join, axis=1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_clf, test_size=0.20, random_state=666, stratify=stratifier
)
X_train, y_train = ros.fit_resample(X_train, y_train)

# Model selection

In [ ]:

MODELS = {
    'xgboost'  : {'lib':'xgboost.XGBClassifier',  'n_trials': 50},
    'lightgbm' : {'lib':'lightgbm.LGBMClassifier', 'n_trials': 50},
    'catboost' : {'lib':'catboost.CatBoostClassifier', 'n_trials': 50}, 
}

def optimise_model(name, lib_path, n_trials=1):

    module_path, cls_name = lib_path.rsplit('.',1)
    cls = __import__(module_path, fromlist=[cls_name]).__dict__[cls_name]

    def objective(trial):

        if name == 'xgboost':
            params = {
                'max_depth'       : trial.suggest_int('max_depth', 3, 15),
                'min_child_weight': trial.suggest_float('min_child_weight', 1e-2, 10.0, log=True),
                'gamma'           : trial.suggest_float('gamma', 0.0, 5.0),                        
                'learning_rate'   : trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
                'n_estimators'    : trial.suggest_int('n_estimators', 100, 1000),              
                'subsample'       : trial.suggest_float('subsample', 0.5, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
                'reg_alpha'       : trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),        
                'reg_lambda'      : trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),        
                'objective'       : 'binary:logistic',
                'eval_metric'     : 'logloss',
                'random_state'    : 666,
                'n_jobs'          : -1,
                'verbosity'       : 0,
            }

        elif name == 'lightgbm':
            params = {
                'boosting_type'   : 'gbdt',
                'max_depth'       : trial.suggest_int('max_depth', 2, 15), 
                'num_leaves'      : trial.suggest_int('num_leaves', 16, 512),
                'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),  
                'min_split_gain'  : trial.suggest_float('min_split_gain', 0.0, 1.0),  
                'learning_rate'   : trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
                'n_estimators'    : trial.suggest_int('n_estimators', 100, 1000),
                'subsample'       : trial.suggest_float('subsample', 0.5, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
                'reg_alpha'       : trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),        
                'reg_lambda'      : trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),         
                'objective'       : 'binary',
                'metric'          : 'binary_logloss',
                'random_state'    : 666,
                'n_jobs'          : -1,
                'verbosity'       : -1,
            }

        else:  # catboost
            params = {
                'depth'           : trial.suggest_int('depth', 3, 15),
                'learning_rate'   : trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
                'iterations'      : trial.suggest_int('iterations', 100, 1000),       
                'l2_leaf_reg'     : trial.suggest_float('l2_leaf_reg', 1, 10, log=True),
                'random_strength' : trial.suggest_float('random_strength', 1e-3, 10.0, log=True),
                'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),    
                'border_count'    : trial.suggest_int('border_count', 32, 255),       
                'loss_function'   : 'Logloss',
                'eval_metric'     : 'Logloss',
                'random_state'    : 666,
                'verbose'         : False,
            }

        model = cls(**params)
        if name=='catboost':    
            model.fit(X_train, y_train, verbose=False)
            preds = model.predict_proba(X_test)[:,1]
        else:
            model.fit(X_train, y_train)
            preds = model.predict_proba(X_test)[:,1]

        return roc_auc_score(y_test, preds)

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials, show_progress_bar=False)
    best_score = study.best_value
    best_params = study.best_params

    if name=='xgboost':
        best_params.update({'objective':'binary:logistic','eval_metric':'logloss',
                            'random_state':666,'n_jobs':-1, 'verbosity': 0})
    elif name=='lightgbm':
        best_params.update({'boosting_type':'gbdt','objective':'binary',
                            'random_state':666,'n_jobs':-1,'verbosity':-1})
    else:
        best_params.update({'random_state':666, 'verbose':False, 'loss_function':'Logloss'})

    final_model = cls(**best_params).fit(X_train, y_train)
    y_pred_bin  = (final_model.predict_proba(X_test)[:,1] >= 0.5).astype(int)

    report = classification_report(y_test, y_pred_bin, output_dict=True, zero_division=0)
    return {
        'model'        : final_model,
        'roc_auc'      : best_score,
        'f1'           : report['1']['f1-score'],
        'accuracy'     : report['accuracy'],
        'best_params'  : best_params,
    }

results = {}
for mname, cfg in MODELS.items():
    print(f'⏳ Optimising {mname}...')
    results[mname] = optimise_model(mname, cfg['lib'], cfg['n_trials'])

import pandas as pd, json, pprint, numpy as np
summary = pd.DataFrame({k:{'ROC-AUC':v['roc_auc'],
                           'F1':v['f1'],
                           'Accuracy':v['accuracy']} for k,v in results.items()}).T
print('\nPerformance (test set):')
display(summary.sort_values('ROC-AUC', ascending=False))

winner = summary['ROC-AUC'].idxmax()
print(f'\n🏆 Best model: **{winner}**')
print('Tuned hyper-parameters:')
pprint.pprint(results[winner]['best_params'])

from importlib import import_module
module_path, cls_name = MODELS[winner]['lib'].rsplit('.',1)
winner_model_cls = import_module(module_path).__dict__[cls_name]
best_params = results[winner]['best_params']

# Leave-one-chromosome-out CV

In [ ]:
df['chrom'] = df['chr_pos_gt'].str.split(':', n=1).str[0].str.replace('chr','', regex=False)
chromosomes = sorted(df['chrom'].unique(), key=lambda x: (not x.isdigit(), int(x) if x.isdigit() else x))

X_full = pd.get_dummies(df[FEATURES], drop_first=True)
y_full = y_clf.values

per_chr_auc = {}
plt.figure(figsize=(6,5))

for chrom in chromosomes:
    train_idx = df['chrom'] != chrom
    test_idx  = df['chrom'] == chrom

    model = winner_model_cls(**best_params)
    model.fit(X_full[train_idx], y_full[train_idx])

    y_prob = model.predict_proba(X_full[test_idx])[:,1]
    auc = roc_auc_score(y_full[test_idx], y_prob)
    per_chr_auc[chrom] = auc

    fpr, tpr, _ = roc_curve(y_full[test_idx], y_prob)
    plt.plot(fpr, tpr, alpha=0.3, label=f'chr{chrom} (AUC {auc:.2f})')

plt.plot([0,1],[0,1],'--')
plt.xlabel('False-positive rate')
plt.ylabel('True-positive rate')
plt.title('Leave-One-Chromosome-Out ROC curves')
plt.legend(fontsize='xx-small', bbox_to_anchor=(1.02,1), loc='upper left')
plt.tight_layout()
plt.show()

print('AUC per chromosome:')
for c, a in per_chr_auc.items():
    print(f'  chr{c:>2}: {a:.3f}')
print(f"\nMean AUC = {np.mean(list(per_chr_auc.values())):.3f}")

In [ ]:
y_true_all, y_prob_all = [], []

for chrom in chromosomes:
    train_idx = df['chrom'] != chrom
    test_idx  = df['chrom'] == chrom

    model = winner_model_cls(**best_params)
    model.fit(X_full[train_idx], y_full[train_idx])

    y_true_all.append(y_full[test_idx])
    y_prob_all.append(model.predict_proba(X_full[test_idx])[:, 1])

    del model; gc.collect()

y_true_all = np.concatenate(y_true_all)
y_prob_all = np.concatenate(y_prob_all)
y_pred_all = (y_prob_all >= 0.5).astype(int)
print(classification_report(y_true_all, y_pred_all, digits=3, zero_division=0))

In [ ]:
model = winner_model_cls(**best_params)
model.fit(X_train, y_train)

if hasattr(model, "feature_importances_"):
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1][:20]
    plt.figure(figsize=(6,4))
    plt.barh(range(len(indices))[::-1], importances[indices])
    plt.yticks(range(len(indices))[::-1], X_train.columns[indices])
    plt.title("Top 20 Feature Importances")
    plt.tight_layout()
    plt.show()
else:
    print("The winning model does not expose feature_importances_.")